### Binarizar las imagenes para poder identificar cual es la del círculo y cual la del cuadrado


In [2]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

In [4]:
#Imagenes que queremos para identificar el circulo y el cuadrado: 
imgs_path = ["Images/pattern_image_4.jpg","Images/pattern_image_10.jpg"]
imgs_pattern = load_images(imgs_path)

C:\Users\mtfra\AppData\Local\Temp\ipykernel_28256\283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [13]:
def load_reference_shapes(ref_images):
    references = {}
    for name, path in ref_images.items():
        # Leer y convertir a escala de grises
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        # Binarizar la imagen
        _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
        # Detectar contornos
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        references[name] = contours[0]  # Asumimos un solo contorno por imagen
    return references

In [14]:
# Comparar contornos detectados en un frame con las referencias
def match_shapes(contour, reference_shapes):
    for name, ref_contour in reference_shapes.items():
        # Usar comparación de contornos (OpenCV matchShapes)
        similarity = cv2.matchShapes(contour, ref_contour, cv2.CONTOURS_MATCH_I1, 0.0)
        if similarity < 0.1:  # Umbral ajustable
            return name  # Retorna el nombre del patrón coincidente
    return None

In [15]:
# Detectar la posición del lápiz en el frame
def detect_pencil_position(frame):
    # Convertir a espacio de color HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Rango de color para el lápiz (ajustar valores según el color del lápiz)
    lower_red = np.array([0, 120, 70])  # Limite inferior de rojo
    upper_red = np.array([10, 255, 255])  # Limite superior de rojo
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Encontrar contornos del lápiz
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Tomar el contorno más grande como lápiz
        pencil_contour = max(contours, key=cv2.contourArea)
        # Calcular el centroide del lápiz
        M = cv2.moments(pencil_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            return (cx, cy)  # Retornar coordenadas del lápiz
    return None

In [17]:
def detect_shapes_in_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar bordes con Sobel
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = cv2.magnitude(sobel_x, sobel_y)

    # Normalizar y binarizar
    sobel_normalized = np.uint8(255 * sobel_combined / np.max(sobel_combined))
    _, binary = cv2.threshold(sobel_normalized, 50, 255, cv2.THRESH_BINARY)

    # Encontrar contornos
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours

In [18]:
def get_shape_center(shape_name, shapes, reference_shapes):
    for contour in shapes:
        matched_shape = match_shapes(contour, reference_shapes)
        if matched_shape == shape_name:
            # Calcular centroide
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                return (cx, cy)  # Coordenadas del centro
    return None

In [ ]:
def process_video_with_phases(video_path, ref_images):
    # Cargar referencias (círculo y cuadrado)
    reference_shapes = load_reference_shapes(ref_images)
    
    # Estado inicial
    current_phase = 0  # Comenzamos fuera de las fases (0 = inicio)
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detectar formas y posición del lápiz
        shapes = detect_shapes_in_frame(frame)
        detected_shapes = [match_shapes(c, reference_shapes) for c in shapes]
        pencil_pos = detect_pencil_position(frame)

        if pencil_pos and "circle" in detected_shapes and "square" in detected_shapes:
            # Obtener posiciones de las figuras
            circle_pos = get_shape_center("circle", shapes, reference_shapes)
            square_pos = get_shape_center("square", shapes, reference_shapes)

            # Calcular coordenadas relativas
            dx, dy = pencil_pos[0] - circle_pos[0], pencil_pos[1] - circle_pos[1]
            circle_radius = int(np.sqrt(cv2.contourArea(shapes[0]) / np.pi))

            x, y, w, h = cv2.boundingRect(shapes[1])
            square_left, square_right = x, x + w
            square_top, square_bottom = y, y + h

            # Determinar la fase actual
            if pencil_pos[0] < min(circle_pos[0], square_pos[0]):
                phase = 1
            elif dx**2 + dy**2 <= circle_radius**2:
                phase = 2
            elif square_left <= pencil_pos[0] <= square_right and square_top <= pencil_pos[1] <= square_bottom:
                phase = 3
            elif pencil_pos[0] > max(circle_pos[0], square_pos[0]):
                phase = 4
            else:
                phase = current_phase  # Mantén la fase si no cambió

            # Validar si la transición es válida
            if phase > current_phase:  # Cambio correcto a la siguiente fase
                print(f"Transición: Fase {current_phase} → Fase {phase}")
                current_phase = phase
            elif phase < current_phase:  # Orden incorrecto
                print(f"¡Fallo! Transición inválida: Fase {current_phase} → Fase {phase}")
                current_phase = -1  # Marcar fallo
                break
            elif phase == current_phase:  # Sin cambio
                print(f"Fase actual: {current_phase}, sin cambio.")

            # Mostrar la fase actual en el frame
            if current_phase > 0:
                cv2.putText(frame, f"Fase: {current_phase}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            else:
                cv2.putText(frame, "Fallo detectado", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        else:
            print("Formas no detectadas o lápiz no encontrado.")

        # Mostrar el frame
        cv2.imshow("Frame", frame)

        # Presionar 'q' para salir
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Resultado final
    if current_phase == 4:
        print("Contraseña correcta")
    else:
        print("Contraseña incorrecta. No se ha pasado efectivamente por los patrones.")